Ethan Zhang ez2262

In [ ]:
# Ethan Zhang ez2262
import numpy as np
from csv import reader
# open file in read mode
d = {}
numrows = 0
keyindex = 0
with open('reviews_tr.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    next(csv_reader)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        numrows+=1
        for word in (row[1]).split(' '):
          if word not in d:
            d[word] = keyindex
            keyindex+=1
unique = len(d)

In [ ]:
# Unigram for training
from scipy.sparse import lil_matrix

tf = lil_matrix((numrows, unique+1))
with open('reviews_tr.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    next(csv_reader)
    curr_row = 0
    for row in csv_reader:
      for word in row[1].split(' '):
        if word in d:
          tf[curr_row, d[word]]+=1
      tf[curr_row, unique] +=1
      curr_row+=1


In [ ]:
# Unigram for test
tf_test = lil_matrix((numrows, unique+1))
with open('reviews_te.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    next(csv_reader)
    # Iterate over each row in the csv using reader object
    curr_row = 0
    for row in csv_reader:
      for word in row[1].split(' '):
        if word in d:
          tf_test[curr_row, d[word]]+=1
      tf_test[curr_row, unique] +=1
      curr_row+=1

In [ ]:
# tf-idf for training
from scipy.sparse import csc_matrix

idf = []
tf_idf = tf.copy()
tmp = tf.tocsc()
for i in range(unique):
  idf.append((tmp.getcol(i)).count_nonzero())

for i in range(numrows):
  for j in range(unique):
    tf_idf[i, j] *= np.log10(numrows/idf[j])

In [ ]:
# tf-idf on test
tf_idf_test = tf_test.copy()
for i in range(numrows):
  for j in range(unique):
    tf_idf_test[i, j] *= np.log10(numrows/idf[j])

In [ ]:
# bigram
d_bigram = {}
key_index_bigram = 0
with open('reviews_tr.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    next(csv_reader)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        # row variable is a list that represents a row in csv
        l = (row[1]).split(' ')
        for i in range(1, len(l)):
          if (l[i-1], l[i]) not in d_bigram:
            d_bigram[(l[i-1], l[i])] = key_index_bigram
            key_index_bigram+=1
num_bigrams = len(d_bigram)

In [ ]:
# bigram on training
bigram = lil_matrix((numrows, num_bigrams+1))
with open('reviews_tr.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    next(csv_reader)
    # Iterate over each row in the csv using reader object
    curr_row = 0
    for row in csv_reader:
      l = (row[1]).split(' ')
      for i in range(1, len(l)):
        if (l[i-1], l[i]) in d_bigram:
          bigram[curr_row, d_bigram[(l[i-1], l[i])]]+=1
      bigram[curr_row, num_bigrams] +=1
      curr_row+=1

In [ ]:
# bigram on test
bigram_test = lil_matrix((numrows, num_bigrams+1))
with open('reviews_te.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    next(csv_reader)
    # Iterate over each row in the csv using reader object
    curr_row = 0
    for row in csv_reader:
      l = (row[1]).split(' ')
      for i in range(1, len(l)):
        if (l[i-1], l[i]) in d_bigram:
          bigram_test[curr_row, d_bigram[(l[i-1], l[i])]]+=1
      bigram_test[curr_row, num_bigrams] +=1
      curr_row+=1

In [ ]:
# Perceptron
from sklearn.utils import shuffle

def perceptron(matrix):

  curr_weight = np.zeros(shape=(matrix.shape[1], 1))
  with open('reviews_tr.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    next(csv_reader)
    curr_row = 0
    for row in csv_reader:
      dot_product = (matrix[curr_row].dot(curr_weight))[0][0]
      if (int(row[0])==0 and dot_product>=0):
        tmp = curr_weight-(matrix[curr_row].reshape(matrix.shape[1], 1))
        curr_weight = np.squeeze(np.asarray(tmp)).reshape(matrix.shape[1], 1)
      elif (int(row[0])==1 and dot_product<0):
        tmp = curr_weight+(matrix[curr_row].reshape(matrix.shape[1], 1))
        curr_weight = np.squeeze(np.asarray(tmp)).reshape(matrix.shape[1], 1)
      curr_row+=1
  
  # pass 2
  total_weight = curr_weight
  matrix = shuffle(matrix)
  with open('reviews_tr.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    next(csv_reader)
    curr_row = 0
    for row in csv_reader:
      dot_product = (matrix[curr_row].dot(curr_weight))[0][0]
      if (int(row[0])==0 and dot_product>=0):
        tmp = curr_weight-(matrix[curr_row].reshape(matrix.shape[1], 1))
        curr_weight = np.squeeze(np.asarray(tmp)).reshape(matrix.shape[1], 1)
      elif (int(row[0])==1 and dot_product<0):
        tmp = curr_weight+(matrix[curr_row].reshape(matrix.shape[1], 1))
        curr_weight = np.squeeze(np.asarray(tmp)).reshape(matrix.shape[1], 1)
      curr_row+=1
  
      
  return 1/(matrix.shape[0]+1) * total_weight
  

In [ ]:
unigram_perceptron = perceptron(tf)

In [ ]:
a = np.squeeze(np.asarray(unigram_perceptron))
indices = np.argsort(a)
print(a)
print(indices[::-1][:10])
top_indices = list(indices[::-1][:10])
worst_indices = list(indices[:10])
print(max(a))
print(min(a))


[-1.9999980e-06  3.9999960e-06 -8.9999910e-06 ...  0.0000000e+00
  0.0000000e+00  1.3999986e-05]
[546   5 135 152 981 347  91 571 185 539]
4.6999953000047e-05
-5.8999941000059004e-05


In [ ]:
top_words = [0]*10
for key in d:
  if d[key] in top_indices:
    top_words[top_indices.index(d[key])] = key
print(top_words)
worst_words = [0]*10
for key in d:
  if d[key] in worst_indices:
    worst_words[worst_indices.index(d[key])] = key
print(worst_words)

['best', 'great', 'love', 'delicious', 'vegas', 'amazing', 'all', 'well', 'friendly', 'your']
['not', 'if', 'no', 'only', 'average', 't', 'bad', 'she', 'came', 'manager']


In [ ]:
def accuracy(weights, test_matrix):
  correct = 0
  with open('reviews_te.csv', 'r') as read_obj:
    csv_reader = reader(read_obj)
    next(csv_reader)
    curr_row = 0
    for row in csv_reader:
      dot_product = (test_matrix[curr_row].dot(weights))[0][0]
      if (dot_product<0 and int(row[0])==0) or (dot_product>=0 and int(row[0])==1):
        correct+=1
      curr_row+=1
  return correct/curr_row


In [ ]:
a = accuracy(unigram_perceptron, tf_test)
print(a)

0.7945751932075896


In [ ]:
unigram_perceptron = perceptron(tf)
tf_idf_perceptron = perceptron(tf_idf_test)
bigram_perceptron = perceptron(bigram_test)

In [ ]:
unigram_accuracy = accuracy(unigram_perceptron, tf_test)
tf_idf_accuracy = accuracy(tf_idf_perceptron, tf_test)
bigram_accuracy = accuracy(bigram_perceptron, tf_test)

In [ ]:
import matplotlib.pyplot as plt
x1 = [500000,650000,800000,1000000]
x2 = [500000,650000,800000,1000000]
x3 = [500000,650000,800000,1000000]
y1 = [0.855, 0.845, 0.872, 0.88]
y2 = [0.84, 0.855, 0.86, 0.87]
y3 = [0.86, 0.868, 0.894, 0.9045]

plt.plot(x1,y1, label="Unigram")
plt.plot(x2,y2, label="tf-idf")
plt.plot(x3,y3, label="Bigram")
plt.legend()
plt.ylabel("Accuracy")
plt.xlabel("Training Size")
plt.title("Unigram vs tf-idf vs Bigram on Training Size w/Original Test Size")
plt.savefig("Train.png")

In [ ]:

x1 = [800000,1000000, 1200000]
x2 = [800000,1000000, 1200000]
x3 = [800000,1000000, 1200000]
y1 = [0.863, 0.88, 0.89]
y2 = [0.862, 0.87, 0.875]
y3 = [0.890, 0.9045, 0.90]

plt.plot(x1,y1, label="Unigram")
plt.plot(x2,y2, label="tf-idf")
plt.plot(x3,y3, label="Bigram")
plt.legend()
plt.ylabel("Accuracy")
plt.xlabel("Training Size")
plt.title("Unigram vs tf-idf vs Bigram on Different Train/Test Splits")
plt.savefig("TrainTest.png")